In [1]:
import numpy as np
import pandas as pd
import os

In [14]:
data_directory = "../Data/Study/"
hololenses = ['A/', 'B/']
intermediate = ''
#subdirectory = "recalibrated_ssHeadConstrained/"

In [3]:
def clean_df(df):
    result = df.loc[(df['Movement'] != "start") & (df['Movement'] != "transition") & (df['EyeTrackingEnabled'] == True) & (df['EyeCalibrationValid'] == True) & (df['EyeTrackingEnabledAndValid'] == True) & (df['EyeTrackingDataValid'] == True) & (df['GazeInputSource'] == "Eyes")].copy()
    #result = result[result['Movement'] != "transition"]
    #result = result[result['EyeTrackingDataValid'] == 'TRUE']
    #result = result[result['GazeInputSource'] == 'Eyes']
    result.reset_index(inplace=True, drop=True)
    return result

In [4]:
def calculate_cosine_error(df):
    df['cosineError'] = df.apply(lambda row: np.degrees(np.arccos(np.dot(np.array((row['EyeGazeDirection.x'], row['EyeGazeDirection.y'], row['EyeGazeDirection.z'])), np.array((row['localTransform.position.x'], row['localTransform.position.y'], row['localTransform.position.z'])))/(np.linalg.norm(np.array((row['EyeGazeDirection.x'], row['EyeGazeDirection.y'], row['EyeGazeDirection.z']))) * np.linalg.norm(np.array((row['localTransform.position.x'], row['localTransform.position.y'], row['localTransform.position.z'])))))), axis=1)
    return df

In [5]:
def calculate_euclidean_error(df):
    #df['euclideanError'] = df.apply(lambda row: np.linalg.norm(np.array((row['GazeAngleX'], row['GazeAngleY']))-np.array((row['TargetAngleX'], row['TargetAngleY']))), axis=1)
    #df['recalibratedEuclideanError'] = df.apply(lambda row: np.linalg.norm(np.array((row['CalibratedGazeAngleX'], row['CalibratedGazeAngleY']))-np.array((row['TargetAngleX'], row['TargetAngleY']))), axis=1)
    df['euclideanError'] = np.sqrt((df.GazeAngleX-df.TargetAngleX) * (df.GazeAngleX-df.TargetAngleX) + (df.GazeAngleY-df.TargetAngleY) * (df.GazeAngleY-df.TargetAngleY))
    df['recalibratedEuclideanError'] = np.sqrt((df.CalibratedGazeAngleX-df.TargetAngleX) * (df.CalibratedGazeAngleX-df.TargetAngleX) + (df.CalibratedGazeAngleY-df.TargetAngleY) * (df.CalibratedGazeAngleY-df.TargetAngleY))
    return df

In [15]:
def calculate_errors(subdirectory):
    output_subdirectory = 'error/' + subdirectory
    for hololens in hololenses:
        participantsList = os.listdir(data_directory + hololens)
        for participant in participantsList:
            print(participant)
            if not participant == '.DS_Store':
                if os.path.exists(data_directory + hololens + participant + '/' + intermediate + output_subdirectory) and len(os.listdir(data_directory + hololens + participant + '/' + intermediate + output_subdirectory)) == 12:
                    print('already calculated')
                    continue
                fileList = os.listdir(data_directory + hololens + participant + '/' + intermediate + subdirectory)
                for file in fileList:
                    if '.csv' in file:
                        filePath = data_directory + hololens + participant + '/' + intermediate
                        df = pd.read_csv(filePath + subdirectory + file)
                        error_df = calculate_euclidean_error(df)
                        error_df = calculate_cosine_error(error_df)
                        if not os.path.exists(filePath + output_subdirectory):
                            os.makedirs(filePath + output_subdirectory)
                        error_df_output_path = filePath + output_subdirectory + '/' + file
                        error_df.to_csv(error_df_output_path, index=False)
                        #print(file)
                        #print("uncalibrated: " + str(error_df['euclideanError'].mean()))
                        #print("calibrated: " + str(error_df['recalibratedEuclideanError'].mean()))
                        #print("cosine: " + str(error_df['cosineError'].mean()))
                        #print("")

In [19]:
calculate_errors("recalibrated_wsWalking/")

03
04
32
34
05
02
.DS_Store
18
27
29
10
19
43
07
36
09
37
08
01
06
39
41
25
35
33
.DS_Store
20
11
16
42
28
17
26
21
38
31
30
24
23
15
12
40
13
14
22
